In [1]:
from data_2c_generators import get_iterator, get_dataset
from classifiers import theme_classifier, personal_classifier
import torch
import torch.nn as nn
from torchtext.vocab import GloVe 
GLOVE_EMBEDDING = GloVe(name="6B", dim=300)

In [2]:
(train_dataset, val_dataset, test_dataset, \
     review_text_FIELD, theme_FIELD, perspective_FIELD) = get_dataset(vectors = GLOVE_EMBEDDING)

In [3]:
batch_size = 20
train_iter = get_iterator(train_dataset, batch_size, train=True, shuffle=True, repeat=False)
val_iter = get_iterator(val_dataset, batch_size, train=False, shuffle=True, repeat=False)
test_iter = get_iterator(test_dataset, batch_size, train=False, shuffle=True, repeat=False)

In [4]:
val_list = list(val_iter)

In [5]:
batch = val_list[500]
x = batch.review_text.transpose(1, 0).int()[:10]
y = batch.theme.int()
z = batch.perspective.int()

for idx in range(x.shape[0]):
    #print(x.shape, y.shape)
    print("{} | {} | {}".format(' '.join([review_text_FIELD.vocab.itos[_] for _ in x[idx]]),
                                          theme_FIELD.vocab.itos[y[idx]],
                                          perspective_FIELD.vocab.itos[z[idx]]))

<sos> their scenes are so slow . <eos> | plot | False
<sos> five stars , do n't miss <eos> | other | False
<sos> great cast , poorly written directed <eos> | acting | False
<sos> great dual protagonist antagonist story . <eos> | plot | False
<sos> boring for the first two hours <eos> | other | False
<sos> trailer was better than movie . <eos> | effect | False
<sos> i liked the whole movie . <eos> | other | True
<sos> touch my heart so amazing <unk> <eos> | other | True
<sos> i say check it out . <eos> | other | True
<sos> i give it a b . <eos> | other | True


In [6]:
print(theme_FIELD.vocab.itos)
print(perspective_FIELD.vocab.itos)

['<unk>', 'other', 'plot', 'acting', 'effect', 'production']
['<unk>', 'False', 'True']


In [7]:
vocab_size = review_text_FIELD.vocab.vectors.shape[0]
label_0_size = len(theme_FIELD.vocab) - 1
label_1_size = len(perspective_FIELD.vocab) - 1
emb_dim = review_text_FIELD.vocab.vectors.shape[1]
vectors = train_dataset.fields["review_text"].vocab.vectors
hidden_dim = 1024
layers = 2
dropout = .5

vocab_size, label_0_size, label_1_size, emb_dim, vectors.shape

(12304, 5, 2, 300, torch.Size([12304, 300]))

In [8]:
from baseline_2_model import BaseModel, repackage_hidden

In [9]:
import numpy as np
import gc

In [10]:
EOS_token = review_text_FIELD.vocab.stoi['<eos>']
EOS_token

3

In [11]:
def forward_pass(model, data, labels, i):
    
    split_tf = data.shape[0] - (i % data.shape[0])
    #print(split_tf)
    
    seq_len = data.shape[0]
    data_tf = data[:split_tf+int(i==0),:]
    data_nf = data[split_tf:,:]

    output_flat = None
    hidden = None
    
    if split_tf > 0:

        data = data_tf
        output_tf, hidden = model(data, labels, hidden)
        repackage_hidden(hidden)
        output_flat = output_tf.contiguous().view(-1, vocab_size)

    if split_tf < seq_len:

        data = data_nf
        shape = tuple((*data_nf.shape, vocab_size))
        output_nf = torch.zeros(shape).cuda()
        hidden_i = None
        data_i = data[0,:]    

        for di in range(data_nf.shape[0]):
            params = data_i.unsqueeze(0), labels, hidden_i
            output_i, hidden_i = model(*params)
            hidden_i = repackage_hidden(hidden_i)
            topv, topi = output_i.topk(1)
            data_i = topi.squeeze().detach()
            output_nf[di,:] = output_i

        temp_output_flat = output_nf.contiguous().view(-1, vocab_size)
        if output_flat is None:
            output_flat = temp_output_flat
        else:
            output_flat = torch.cat([output_flat, temp_output_flat], 0)
            
    return output_flat

In [12]:
def transtaltion2string(raw_translations, target_vocab = review_text_FIELD, max_words=30000):
    string_translations = []
    for raw_sentence in raw_translations:
        string_sentence = []
        for i, word_idx in enumerate(raw_sentence):
            if i == max_words: break
            word = target_vocab.vocab.itos[word_idx]
            if word != '<sos>':
                string_sentence.append(word)
        string_translations.append(string_sentence)
    return [' '.join(_) for _ in string_translations]

In [13]:
def get_predicted_labels(output_flat, original_shape, ground_truth):
    topv, topi = output_flat.topk(1, -1)
    topi = topi.view(*original_shape)
    
    theme_predictions = []
    perspective_predictions = []
    for i in range(topi.shape[1]):
        text = transtaltion2string([topi[:,i]])[0]
        predicted_theme = theme_classifier(text)
        predicted_perspective = personal_classifier(text)
        
        idx_theme = theme_FIELD.vocab.stoi[predicted_theme] - 1 
        idx_perspective = perspective_FIELD.vocab.stoi[predicted_perspective] - 1 
        theme_predictions.append(idx_theme)
        perspective_predictions.append(idx_perspective)
        
    theme_predictions = torch.tensor(theme_predictions).cuda()
    perspective_predictions = torch.tensor(perspective_predictions).cuda()
    
    classifier_theme_output = (theme_predictions == ground_truth[0]).float()
    classifier_perspective_output = (perspective_predictions == ground_truth[1]).float()
    #acc_score = classifier_output.mean().item()
    
    classifier_output = ((classifier_theme_output + classifier_perspective_output) == 2).float()
    
    return classifier_output

In [14]:
def evaluate(model, data_source, criterion, teacher_forcing = False):
    model.eval()
    total_loss_e = 0
    total_number_of_words = 0
    
    with torch.no_grad():
        for i, batch in enumerate(data_source):
            labels_0 = batch.theme.cuda().long() - 1
            labels_1 = batch.perspective.cuda().long() - 1
            labels = (labels_0, labels_1)
            batch = batch.review_text.cuda().long()
            
            if batch.shape[0] > 3:
                data, targets = batch[1:-1,:], batch[2:,:]
                target_flat = targets.contiguous().view(-1)
                
                tf = i if not teacher_forcing else 0
                output_flat = forward_pass(model, data, labels, tf)
                classifier_output = get_predicted_labels(output_flat, targets.shape, labels)
                
                batch_loss = criterion(output_flat, target_flat, classifier_output, targets.shape).detach().item()
                
                number_of_words = data.shape[0] * data.shape[1]
                total_loss_e += batch_loss * number_of_words
                total_number_of_words += number_of_words
            
    #print(total_loss_e, total_number_of_words)
    return total_loss_e / total_number_of_words

In [31]:
def train(model, ep0, epN, train_iter, dev_iter, optimizer, criterion, 
          max_grad_norm, model_name, best_ppl = float('inf'), teacher_forcing = False):
    
    best_ppl = best_ppl
    
    len_train_iter = len(train_iter)
    for epoch in range(ep0, epN):
        model.train()
        total_loss_e = 0
        total_number_of_words = 0 
        
        for i, batch in enumerate(train_iter):

            labels_0 = batch.theme.cuda().long() - 1
            labels_1 = batch.perspective.cuda().long() - 1
            labels = (labels_0, labels_1)
            batch = batch.review_text.cuda().long()
            hidden = None
            
            if batch.shape[0] > 3:
                data, targets = batch[1:-1,:], batch[2:,:]
                
                tf = i if not teacher_forcing else 0
                output_flat = forward_pass(model, data, labels, tf)
                
                target_flat = targets.contiguous().view(-1)
                classifier_output = get_predicted_labels(output_flat, targets.shape, labels)
                batch_loss = criterion(output_flat, target_flat, classifier_output, targets.shape)
                
                torch.nn.utils.clip_grad_norm_(model.parameters(), max_grad_norm)
                optimizer.zero_grad()
                batch_loss.backward()
                optimizer.step()

                number_of_words = data.shape[0] * data.shape[1]
                total_loss_e += batch_loss.detach().item() * number_of_words
                total_number_of_words += number_of_words
            
                
                if i % 500 == 0:
                    cur_loss = batch_loss.detach().item() 
                    tr_ppl_print = np.exp(cur_loss)
                    print("| epoch {:3d} | batch {} / {} | train_loss {} | train_ppl {}".format(
                            epoch, i, len_train_iter, 
                            np.round(cur_loss, 3), np.round(tr_ppl_print, 3)))

                
                if i % 4999 == 1: #len_train_iter - 1:
                    cur_loss = batch_loss.detach().item()
                    tr_ppl_print = np.exp(cur_loss)
                    gc.collect()
                    val_loss_eval = evaluate(model, dev_iter, criterion_1, teacher_forcing)
                    val_ppl_print = np.exp(val_loss_eval)
                    
                    template = "| epoch {:3d} | batch {} / {} | train_loss {} | train_ppl {} | val_loss {} | val_ppl {}"
                    print(template.format(
                            epoch, i, len_train_iter, 
                            np.round(cur_loss, 3), np.round(tr_ppl_print, 3), 
                            np.round(val_loss_eval, 3), np.round(val_ppl_print, 3)))

                    if val_ppl_print < best_ppl :
                        print('old best ppl {} new best ppl {}'.format(best_ppl, val_ppl_print))
                        best_ppl = val_ppl_print
                        best_model_name = '{}_{}.model'.format(model_name, best_ppl)
                        print('save model...', best_model_name)
                        with open(best_model_name, 'wb') as file:
                            torch.save(model, file) 

                    gc.collect()
                    model.train()
                    
                if i == 40000: break

In [39]:
with open('./baseline_2c/best_model_65.49184179009306.model', 'rb') as file:
    model = torch.load(file)
    model.eval()

In [40]:
model

BaseModel(
  (drop): Dropout(p=0.5)
  (word_embedding): Embedding(12304, 300)
  (label_embedding): Embedding(7, 20)
  (rnn): LSTM(340, 1024, num_layers=2, dropout=0.5)
  (decoder): Linear(in_features=1024, out_features=12304, bias=True)
)

In [41]:
learning_rate = 0.001

optimizer = torch.optim.Adadelta(model.parameters(), lr=learning_rate)#, betas=(.9999, .9999))


def criterion(predictions, targets, labels, targets_shape):
    """
    labels: [1, 0, 0, 1, ...] 1: hit, 0: miss
    """
    c = nn.CrossEntropyLoss(reduction='none',
                           ignore_index=train_dataset.fields["review_text"].vocab.stoi['<pad>']).cuda()
    out = c(predictions, targets)
    out = (out.view(*targets_shape) * (1 - labels)).mean()
    return out

def criterion_1(predictions, targets, labels, targets_shape):
    c = nn.CrossEntropyLoss(reduction='none',
                           ignore_index=train_dataset.fields["review_text"].vocab.stoi['<pad>']).cuda()
    out = c(predictions, targets)
    out = out.mean()
    return out

In [42]:
evaluate(model, test_iter, criterion_1, teacher_forcing=True)

4.1639207230429465

In [43]:
np.exp(4.1639207230429465)

64.32322239515557

In [44]:
train(model,
      ep0 = 0,
      epN = 1,
      train_iter = train_iter,
      dev_iter = val_iter,
      optimizer = optimizer,
      criterion = criterion,
      max_grad_norm = 10,
      model_name = 'no-teacher-forcing/model_ppl_2c_',
      best_ppl = float('inf'),
      teacher_forcing = True)

| epoch   0 | batch 500 / 35138 | train_loss 1.744 | train_ppl 5.72
| epoch   0 | batch 1000 / 35138 | train_loss 2.247 | train_ppl 9.459
| epoch   0 | batch 1500 / 35138 | train_loss 2.44 | train_ppl 11.478
| epoch   0 | batch 2000 / 35138 | train_loss 3.378 | train_ppl 29.315
| epoch   0 | batch 2500 / 35138 | train_loss 3.422 | train_ppl 30.622
| epoch   0 | batch 3000 / 35138 | train_loss 3.328 | train_ppl 27.874
| epoch   0 | batch 3500 / 35138 | train_loss 3.603 | train_ppl 36.72
| epoch   0 | batch 4000 / 35138 | train_loss 3.345 | train_ppl 28.361
| epoch   0 | batch 4500 / 35138 | train_loss 4.058 | train_ppl 57.867
| epoch   0 | batch 5000 / 35138 | train_loss 4.052 | train_ppl 57.491
| epoch   0 | batch 5000 / 35138 | train_loss 4.052 | train_ppl 57.491 | val_loss 4.169 | val_ppl 64.683
old best ppl inf new best ppl 64.6825488006041
save model... no-teacher-forcing/model_ppl_2c__64.6825488006041.model
| epoch   0 | batch 5500 / 35138 | train_loss 3.454 | train_ppl 31.62
| ep

In [45]:
train(model,
      ep0 = 1,
      epN = 2,
      train_iter = train_iter,
      dev_iter = val_iter,
      optimizer = optimizer,
      criterion = criterion,
      max_grad_norm = 10,
      model_name = 'no-teacher-forcing/model_ppl_2c_',
      best_ppl = float('inf'),
      teacher_forcing = True)

| epoch   1 | batch 500 / 35138 | train_loss 1.625 | train_ppl 5.078
| epoch   1 | batch 1000 / 35138 | train_loss 2.227 | train_ppl 9.274
| epoch   1 | batch 1500 / 35138 | train_loss 2.305 | train_ppl 10.029
| epoch   1 | batch 2000 / 35138 | train_loss 3.176 | train_ppl 23.939
| epoch   1 | batch 2500 / 35138 | train_loss 3.326 | train_ppl 27.825
| epoch   1 | batch 3000 / 35138 | train_loss 3.421 | train_ppl 30.596
| epoch   1 | batch 3500 / 35138 | train_loss 3.611 | train_ppl 37.007
| epoch   1 | batch 4000 / 35138 | train_loss 3.2 | train_ppl 24.531
| epoch   1 | batch 4500 / 35138 | train_loss 3.981 | train_ppl 53.547
| epoch   1 | batch 5000 / 35138 | train_loss 3.901 | train_ppl 49.449
| epoch   1 | batch 5000 / 35138 | train_loss 3.901 | train_ppl 49.449 | val_loss 4.138 | val_ppl 62.665
old best ppl inf new best ppl 62.664970197613926
save model... no-teacher-forcing/model_ppl_2c__62.664970197613926.model
| epoch   1 | batch 5500 / 35138 | train_loss 3.462 | train_ppl 31.88

In [46]:
train(model,
      ep0 = 2,
      epN = 4,
      train_iter = train_iter,
      dev_iter = val_iter,
      optimizer = optimizer,
      criterion = criterion,
      max_grad_norm = 10,
      model_name = 'no-teacher-forcing/model_ppl_2c_',
      best_ppl = float('inf'),
      teacher_forcing = True)

| epoch   2 | batch 500 / 35138 | train_loss 1.541 | train_ppl 4.671
| epoch   2 | batch 1000 / 35138 | train_loss 2.224 | train_ppl 9.248
| epoch   2 | batch 1500 / 35138 | train_loss 2.335 | train_ppl 10.326
| epoch   2 | batch 2000 / 35138 | train_loss 3.281 | train_ppl 26.596
| epoch   2 | batch 2500 / 35138 | train_loss 3.462 | train_ppl 31.878
| epoch   2 | batch 3000 / 35138 | train_loss 3.373 | train_ppl 29.157
| epoch   2 | batch 3500 / 35138 | train_loss 3.611 | train_ppl 37.021
| epoch   2 | batch 4000 / 35138 | train_loss 3.279 | train_ppl 26.552
| epoch   2 | batch 4500 / 35138 | train_loss 3.98 | train_ppl 53.498
| epoch   2 | batch 5000 / 35138 | train_loss 3.987 | train_ppl 53.882
| epoch   2 | batch 5000 / 35138 | train_loss 3.987 | train_ppl 53.882 | val_loss 4.122 | val_ppl 61.707
old best ppl inf new best ppl 61.70748352585333
save model... no-teacher-forcing/model_ppl_2c__61.70748352585333.model
| epoch   2 | batch 5500 / 35138 | train_loss 3.362 | train_ppl 28.855

KeyboardInterrupt: 

In [ ]:
train(model,
      ep0 = 4,
      epN = 6,
      train_iter = train_iter,
      dev_iter = val_iter,
      optimizer = optimizer,
      criterion = criterion,
      max_grad_norm = 10,
      model_name = 'no-teacher-forcing/model_ppl_2c_',
      best_ppl = float('inf'),
      teacher_forcing = True)

In [47]:
from baseline_model import BaseModel, repackage_hidden
with open('./no-teacher-forcing/model_ppl_2c__61.58480131769885.model', 'rb') as file:
    model = torch.load(file)
    model.eval()

In [48]:
def criterion(predictions, targets, labels, targets_shape):
    """
    labels: [1, 0, 0, 1, ...] 1: hit, 0: miss
    """
    c = nn.CrossEntropyLoss(reduction='none',
                           ignore_index=train_dataset.fields["review_text"].vocab.stoi['<pad>']).cuda()
    out = c(predictions, targets).mean()
    return out
evaluate(model, test_iter, criterion, teacher_forcing=True)

4.115639458389707

In [49]:
np.exp(4.115639458389707)

61.291395030950795